## Problem 4: What is the closest shopping center from your home / work? (5 points)

In the last problem you should find out the closest shopping center from two different locations a) your home and b) work place. 

**Steps**:

 - Create a txt-file called `activity_locations.txt` (use same formatting as in Problem 1) with two columns:
    - `id`: unique id for each row
    - `addr`:  address of your work and home (or any two addresses in the Helsinki Region.)
    
Save the text file into this repository.
    
 - Read those addresses using pandas and geocode the addresses.
 - Find out the nearest shopping center to these points using Shapely `nearest_points`.
 - Print out the name of the shopping center that is nearest to a) home and b) work. For example: `Shopping center closest to home: REDI`.

In [1]:
# read in input files
import pandas as pd
import geopandas as gpd

locations = pd.read_csv('activity_locations.txt', sep=';')
shopping_centers = gpd.read_file('shopping_centers.shp')

In [2]:
# Geocode activity locations
from geopandas.tools import geocode

geo_locations = geocode(locations['addr'], provider='nominatim', user_agent='autogis_00', timeout=5)
geo_locations['category'] = ['home','work']
geo_locations

,geometry,address,category
0,POINT (25.01811 60.16924),"Koirasaarentie, Hopealaakso, Kruunuvuorenranta...",home
1,POINT (24.96214 60.20501),"Kumpulan kampuskirjasto, 2, Gustaf Hällströmin...",work


In [3]:
# Check coordinate reference system information (re-project/define if needed)
geo_locations = geo_locations.to_crs(shopping_centers.crs)
geo_locations.crs

<Derived Projected CRS: EPSG:3879>
Name: ETRS89 / GK25FIN
Axis Info [cartesian]:
- N[north]: Northing (metre)
- E[east]: Easting (metre)
Area of Use:
- name: Finland - nominally onshore between 24°30'E and 25°30'E but may be used in adjacent areas if a municipality chooses to use one zone over its whole extent.
- bounds: (24.5, 59.94, 25.5, 68.9)
Coordinate Operation:
- name: Finland Gauss-Kruger zone 25
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989 ensemble
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [4]:
# Find the closest shopping center for each activity location (closest shopping center from
# and closest shopping center from work) and print out the results:
from shapely.ops import nearest_points
from shapely.geometry import MultiPoint

nearest_tuples = geo_locations['geometry'].apply(lambda x: nearest_points(x, MultiPoint(shopping_centers['geometry'])), 1).values
for cat, nearest_tuple in zip(geo_locations['category'].values,nearest_tuples):
    origin, dest = nearest_tuple
    print(f'Shopping center closest to {cat}: {shopping_centers[shopping_centers["geometry"]==dest]["name"].iloc[0]}')

Shopping center closest to home: REDI
Shopping center closest to work: Tripla


In [5]:
## Same computation using geopandas sjoin_nearest method

nearest_join = geo_locations.sjoin_nearest(shopping_centers)

for idx, row in nearest_join.iterrows():
    print(f"Shopping center closest to {row['category']}: {row['name']}")

Shopping center closest to home: REDI
Shopping center closest to work: Tripla


Can you think of other application cases for the nearest neighbour analysis?